Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Do the [Plotly Dash](https://dash.plot.ly/) Tutorial, Parts 1 & 2.
- [ ] Add your own stretch goal(s) !

In [0]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module1')

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv('../data/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [4]:
#Number of rows after removing the outliers is
df.shape
#down by 535 rows

(48817, 34)

In [5]:
#checking for Nulls
df.isnull().sum()

bathrooms                  0
bedrooms                   0
created                    0
description             1425
display_address          133
latitude                   0
longitude                  0
price                      0
street_address            10
interest_level             0
elevator                   0
cats_allowed               0
hardwood_floors            0
dogs_allowed               0
doorman                    0
dishwasher                 0
no_fee                     0
laundry_in_building        0
fitness_center             0
pre-war                    0
laundry_in_unit            0
roof_deck                  0
outdoor_space              0
dining_room                0
high_speed_internet        0
balcony                    0
swimming_pool              0
new_construction           0
terrace                    0
exclusive                  0
loft                       0
garden_patio               0
wheelchair_access          0
common_outdoor_space       0
dtype: int64

In [0]:
#Adding the following new featrues
df['bath_bed']=df['bathrooms']+df['bedrooms'] #Number of bathrooms plus bedrooms

In [33]:
df.dtypes

bathrooms                      float64
bedrooms                         int64
created                 datetime64[ns]
description                     object
display_address                 object
latitude                       float64
longitude                      float64
price                            int64
street_address                  object
interest_level                   int64
elevator                         int64
cats_allowed                     int64
hardwood_floors                  int64
dogs_allowed                     int64
doorman                          int64
dishwasher                       int64
no_fee                           int64
laundry_in_building              int64
fitness_center                   int64
pre-war                          int64
laundry_in_unit                  int64
roof_deck                        int64
outdoor_space                    int64
dining_room                      int64
high_speed_internet              int64
balcony                  

In [8]:
#Converting interest level into integer or rather cardinal
df['interest_level'].value_counts()

low       33946
medium    11181
high       3690
Name: interest_level, dtype: int64

In [0]:
df['interest_level'] = df['interest_level'].replace({'low':1,'medium':2,'high':3}).astype('int')

In [10]:
df['interest_level'].value_counts()

1    33946
2    11181
3     3690
Name: interest_level, dtype: int64

In [0]:
#Chaging created feature datetime
df['created'] = pd.to_datetime(df['created'],infer_datetime_format=True)

In [15]:
#Getting the boolean columns and adding them to perk columns
perks_columns = []
for col in df:
  if(df[col].nunique()==2):
    perks_columns.append(col)
print(len(perks_columns))
perks_columns

24


['elevator',
 'cats_allowed',
 'hardwood_floors',
 'dogs_allowed',
 'doorman',
 'dishwasher',
 'no_fee',
 'laundry_in_building',
 'fitness_center',
 'pre-war',
 'laundry_in_unit',
 'roof_deck',
 'outdoor_space',
 'dining_room',
 'high_speed_internet',
 'balcony',
 'swimming_pool',
 'new_construction',
 'terrace',
 'exclusive',
 'loft',
 'garden_patio',
 'wheelchair_access',
 'common_outdoor_space']

In [0]:
#Getting the feature of number of perks for a place
df['no_of_perks']=df[perks_columns].sum(axis=1)

In [0]:
#Getting the bed to bath ratio
#df['bed_per_bathrooms']=df['bedrooms']/df['bathrooms']
#Removing this columns as some values is inf and I currently do not know how to address the inf values

In [26]:
#Now splitting the date into train and test based on created date
# April and May are Train data
#June is the test data
df['created'].describe()

count                   48817
unique                  48148
top       2016-05-14 01:11:03
freq                        3
first     2016-04-01 22:12:41
last      2016-06-29 21:41:47
Name: created, dtype: object

In [30]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,bath_bed,no_of_perks,bed_per_bathrooms
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,0,2.0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,1,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,5,2.0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,3,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,3,1.0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2,1.0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,1,4.0


In [31]:
train = df[df['created']<='2016-05-31']
test= df[df['created']>'2016-05-31']
train.shape,test.shape

((31839, 37), (16978, 37))

In [37]:
#features are the columns which are either float or int
features = [col for col in df if ((df[col].dtype=='float64') |(df[col].dtype=='int64'))]
print(len(features))
features
#Removing price from the features as it is the target
features.remove('price')
features

33


['bathrooms',
 'bedrooms',
 'latitude',
 'longitude',
 'interest_level',
 'elevator',
 'cats_allowed',
 'hardwood_floors',
 'dogs_allowed',
 'doorman',
 'dishwasher',
 'no_fee',
 'laundry_in_building',
 'fitness_center',
 'pre-war',
 'laundry_in_unit',
 'roof_deck',
 'outdoor_space',
 'dining_room',
 'high_speed_internet',
 'balcony',
 'swimming_pool',
 'new_construction',
 'terrace',
 'exclusive',
 'loft',
 'garden_patio',
 'wheelchair_access',
 'common_outdoor_space',
 'bath_bed',
 'no_of_perks',
 'bed_per_bathrooms']

In [0]:
features.remove('bed_per_bathrooms')

In [60]:
#Now training the model on train data on Linear Regression model
from sklearn.linear_model import LinearRegression
model = LinearRegression()
#Fitting the data on train data
model.fit(train[features],train['price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [62]:
print(f'The intercept of the model is {model.intercept_:,.0f}')
coefficients = pd.Series(model.coef_,features)
print(coefficients.to_string())

The intercept of the model is -1,022,279
bathrooms              -8.070868e+14
bedrooms               -8.070868e+14
latitude                1.239787e+03
longitude              -1.315102e+04
interest_level         -4.106630e+02
elevator                1.126883e+02
cats_allowed           -6.653032e+01
hardwood_floors        -1.821686e+02
dogs_allowed            4.084955e+01
doorman                 3.922221e+02
dishwasher              1.916807e+01
no_fee                 -1.376164e+02
laundry_in_building    -1.425560e+02
fitness_center          8.001791e+01
pre-war                -1.255008e+02
laundry_in_unit         4.435357e+02
roof_deck              -2.010020e+02
outdoor_space          -1.152520e+02
dining_room             2.257500e+02
high_speed_internet    -3.284609e+02
balcony                -7.808691e+01
swimming_pool           2.210352e+01
new_construction       -1.682041e+02
terrace                 1.456943e+02
exclusive               1.119918e+02
loft                    1.007789e+

In [72]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
#Based on model predicting the price for both train  and test data
y_train = model.predict(train[features])
y_test = model.predict(test[features])
train_price = train['price']
test_price = test['price']
rmse_train = np.sqrt(mean_squared_error(train_price,y_train))
rmse_test = np.sqrt(mean_squared_error(test_price,y_test))
#MAE for both train and test data
print(f'The MAE for Train :{mean_absolute_error(train_price,y_train):.2f} and Test:{mean_absolute_error(test_price,y_test):.2f}')
print(f'The R2 score for Train :{r2_score(train_price,y_train):.2f} and Test:{r2_score(test_price,y_test):.2f}')
print(f'The RMSE score for Train :{rmse_train:.2f} and Test:{rmse_test:.2f}')


The MAE for Train :673.48 and Test:677.01
The R2 score for Train :0.64 and Test:0.65
The RMSE score for Train :1062.24 and Test:1047.12


In [73]:
#Organising the code
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
# Read New York City apartment rental listing data
df = pd.read_csv('../data/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]
df['bath_bed']=df['bathrooms']+df['bedrooms'] #Number of bathrooms plus bedrooms
#Converting interest level into integer or rather cardinal
df['interest_level'] = df['interest_level'].replace({'low':1,'medium':2,'high':3}).astype('int')
#Chaging created feature datetime
df['created'] = pd.to_datetime(df['created'],infer_datetime_format=True)
#Getting the boolean columns and adding them to perk columns
perks_columns = []
for col in df:
  if(df[col].nunique()==2):
    perks_columns.append(col)
#Getting the feature of number of perks for a place
df['no_of_perks']=df[perks_columns].sum(axis=1)
features = [col for col in df if ((df[col].dtype=='float64') |(df[col].dtype=='int64'))]
#Removing price from the features as it is the target
features.remove('price')
train = df[df['created']<='2016-05-31']
test= df[df['created']>'2016-05-31']
model = LinearRegression()
#Fitting the data on train data
model.fit(train[features],train['price'])
print(f'The intercept of the model is {model.intercept_:,.0f}')
coefficients = pd.Series(model.coef_,features)
print(coefficients.to_string())
#Based on model predicting the price for both train  and test data
y_train = model.predict(train[features])
y_test = model.predict(test[features])
train_price = train['price']
test_price = test['price']
rmse_train = np.sqrt(mean_squared_error(train_price,y_train))
rmse_test = np.sqrt(mean_squared_error(test_price,y_test))
#MAE for both train and test data
print(f'The MAE for Train :{mean_absolute_error(train_price,y_train):.2f} and Test:{mean_absolute_error(test_price,y_test):.2f}')
print(f'The R2 score for Train :{r2_score(train_price,y_train):.2f} and Test:{r2_score(test_price,y_test):.2f}')
print(f'The RMSE score for Train :{rmse_train:.2f} and Test:{rmse_test:.2f}')


The intercept of the model is -1,022,279
bathrooms              -8.070868e+14
bedrooms               -8.070868e+14
latitude                1.239787e+03
longitude              -1.315102e+04
interest_level         -4.106630e+02
elevator                1.126883e+02
cats_allowed           -6.653032e+01
hardwood_floors        -1.821686e+02
dogs_allowed            4.084955e+01
doorman                 3.922221e+02
dishwasher              1.916807e+01
no_fee                 -1.376164e+02
laundry_in_building    -1.425560e+02
fitness_center          8.001791e+01
pre-war                -1.255008e+02
laundry_in_unit         4.435357e+02
roof_deck              -2.010020e+02
outdoor_space          -1.152520e+02
dining_room             2.257500e+02
high_speed_internet    -3.284609e+02
balcony                -7.808691e+01
swimming_pool           2.210352e+01
new_construction       -1.682041e+02
terrace                 1.456943e+02
exclusive               1.119918e+02
loft                    1.007789e+

In [45]:
# Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
# Engineer at least two new features. (See below for explanation & ideas.)
# Fit a linear regression model with at least two features.
# Get the model's coefficients and intercept.
# Get regression metrics RMSE, MAE, and  R2 , for both the train and test data.
# What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
# As always, commit your notebook to your fork of the GitHub repo.


bathrooms               0
bedrooms                0
latitude                0
longitude               0
interest_level          0
elevator                0
cats_allowed            0
hardwood_floors         0
dogs_allowed            0
doorman                 0
dishwasher              0
no_fee                  0
laundry_in_building     0
fitness_center          0
pre-war                 0
laundry_in_unit         0
roof_deck               0
outdoor_space           0
dining_room             0
high_speed_internet     0
balcony                 0
swimming_pool           0
new_construction        0
terrace                 0
exclusive               0
loft                    0
garden_patio            0
wheelchair_access       0
common_outdoor_space    0
bath_bed                0
no_of_perks             0
dtype: int64